In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 24, 3, padding=1) #28
        self.bn1 = nn.BatchNorm2d(24)
        self.dp1 = nn.Dropout(0.05)
        self.conv2 = nn.Conv2d(24, 20, 3, padding=1)#28
        self.bn2 = nn.BatchNorm2d(20)
        self.dp2 = nn.Dropout(0.05)
        self.conv3 = nn.Conv2d(20, 15, 3, padding=1)#28
        self.bn3 = nn.BatchNorm2d(15)
        self.dp3 = nn.Dropout(0.05)
        self.pool1 = nn.MaxPool2d(2, 2)#14
        
        self.conv4 = nn.Conv2d(15, 24, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(24)
        self.dp4 = nn.Dropout(0.05)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(24, 20, 3)
        self.bn5 = nn.BatchNorm2d(20)
        self.dp5 = nn.Dropout(0.05)
        
        self.conv6 = nn.Conv2d(20, 14, 3)
        self.bn6 = nn.BatchNorm2d(14)
        self.dp6 = nn.Dropout(0.05)
        self.conv7 = nn.Conv2d(14, 10, 3)

    def forward(self, x):
        x = F.relu(self.dp2((self.bn2(self.conv2(F.relu(self.dp1(self.bn1(self.conv1(x)))))))))
        x=  self.pool1(F.relu(self.dp3(self.bn3(self.conv3(x)))))
        x = self.pool2(self.dp4(self.bn4(F.relu(self.conv4(x)))))
        x = F.relu(self.dp5(self.bn5(self.conv5(x))))
        x = F.relu(self.dp6(self.bn6(self.conv6(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 28, 28]             240
       BatchNorm2d-2           [-1, 24, 28, 28]              48
           Dropout-3           [-1, 24, 28, 28]               0
            Conv2d-4           [-1, 20, 28, 28]           4,340
       BatchNorm2d-5           [-1, 20, 28, 28]              40
           Dropout-6           [-1, 20, 28, 28]               0
            Conv2d-7           [-1, 15, 28, 28]           2,715
       BatchNorm2d-8           [-1, 15, 28, 28]              30
           Dropout-9           [-1, 15, 28, 28]               0
        MaxPool2d-10           [-1, 15, 14, 14]               0
           Conv2d-11           [-1, 24, 14, 14]           3,264
      BatchNorm2d-12           [-1, 24, 14, 14]              48
    

<ipython-input-2-ad17b7fe7da1>:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:00<00:00, 412056842.00it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 91839040.05it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 166411400.58it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 2441749.39it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
for epoch in range(1, 21):
    print(f'Epoch {epoch}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch 1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-ad17b7fe7da1>:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.04950754716992378 batch_id=468: 100%|██████████| 469/469 [00:26<00:00, 17.42it/s]



Test set: Average loss: 0.0470, Accuracy: 9871/10000 (98.71%)

Epoch 2


loss=0.04106895998120308 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.58it/s]



Test set: Average loss: 0.0410, Accuracy: 9886/10000 (98.86%)

Epoch 3


...


Test set: Average loss: 0.0172, Accuracy: 9946/10000 (99.46%)

Epoch 19


loss=0.004187766928225756 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 21.05it/s]



Test set: Average loss: 0.0180, Accuracy: 9944/10000 (99.44%)

Epoch 20


loss=0.0013504914240911603 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.99it/s]



Test set: Average loss: 0.0187, Accuracy: 9941/10000 (99.41%)

